# Minimum Spanning Tree Streaming Algorithm

In this notebook we present the method introduced in the paper <a href="http://cmm.ensmp.fr/~marcoteg/cv/publi_pdf/gigli/20180525_ICIP18_StreamingMst.pdf">ON MINIMUM SPANNING TREE STREAMING FOR IMAGE ANALYSIS.</a>

### Context

Let $\mathcal{I}_t$ be an image streaming during time, in which new pixels come from one side of the image. At each interval of time $t=0,\ldots, T$, a new block $I_t$ of pixels arrives, such that $\mathcal{I}_t = \mathcal{I}_{t-1} \cup I_t$. To simplify our problem, we assume that the new image $I_t$ shares a column with $\mathcal{I}_{t-1}$, i.e. the last column of $\mathcal{I}_{t-1}$ is the first column of $I_t$, as in the following figure:
<img style="height: 150px; margin-top: 20px; margin-bottom: 20px", src="fig/img_1.png">

### Minimum spanning tree streaming

In this context, the method that we present in this notebook addresses the problem of computing a minimum spanning tree of the image $\mathcal{I}_t$, at each iteration $t=0,\ldots,T$, using the information of the itearation $t-1$.

In [ ]:
# loading libraries
from scipy.misc import imread
from scipy.sparse.csgraph import minimum_spanning_tree
from matplotlib import pyplot as plt


import loadlib

from SST.utils import plot_graph, stick_two_images
from SST.streaming import streaming_spanning_tree
from SST.streaming.streaming_generators import HorizontalStreaming

%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
test_img = imread('RGB_US-CA-SanDiego_2010_05_03.tif')

# big image that we are going to split in blocks
I = test_img[300:400,:199]

plt.figure(figsize=(16,8))
plt.imshow(I)
plt.axis('off')
plt.show()

### HorizontalStreaming Generator

We show a stream generator that fit to the context that we described before. This generator yields at each iteration the following four elements:

- ith block image
- the 4-connected graph associated to the ith block image
- id of a particular node to use as root node, (id of top-right pixel of the ith img)
- ids of nodes in the frontier between ith image and (i+1)th image

In [ ]:
# big image that we are going to split in blocks
I = test_img[300:400,:199]

gen = HorizontalStreaming(I)
stream = gen.generate_stream(block_shape=(100,100))

all_graphs = []
for n, (img, g, r, front) in enumerate(stream):
    if n == 0:
        total_img = img
    else:
        total_img = stick_two_images(total_img, img, num_overlapping=1, direction='H')
    
    if front is not None:
        print("{} is the root node id for the mst at iteration {}".format(r, n))
        print("{} are the first 10 nodes in the frontier at iteration {}".format(front[:10], n))
        
    all_graphs.append(g)
    plot_graph(total_img, all_graphs, order='F', colors=['b', 'r'], figsize=(8*(n+1),8))

### Proposed method

As we have shown, at each iteration the stream generator yields, among all the nodes in the frontiers between the ith image and the one in the successive iteration.



In [ ]:
I = test_img[300:400,:298]

gen = HorizontalStreaming(I)
stream = gen.generate_stream(block_shape=(100,100))


for n, (t, e, img) in enumerate(streaming_spanning_tree(stream, return_img=True)):
    if n == 0:
        total_img = img
    else:
        total_img = stick_two_images(total_img, img, num_overlapping=1, direction='H')
    if e is not None:
        plot_graph(total_img, [t,e], order='F', colors=['b', 'r'], figsize=(8*(n+1),8))
    else:
        plot_graph(total_img, t, order='F', colors=['b'], figsize=(8*(n+1),8))

### MST Decomposition

In [ ]:
from SST.utils import resize_graph
I = test_img[150:200, 250:398]
gen = HorizontalStreaming(I)
stream = gen.generate_stream(block_shape=(50,50))
T = None
for n, (t, e, img) in enumerate(streaming_spanning_tree(stream, return_img=True)):
    if n == 0:
        total_img = img
        T = t
    else:
        total_img = stick_two_images(total_img, img, num_overlapping=1, direction='H')
    if e is not None:
        if n == 0:
            plot_graph(total_img, [t,e], order='F', colors=['C2', 'r'], saveplot=True, title='dec'+str(n))
        else:
            plot_graph(total_img, [t, e], order='F', colors=['C2', 'r'], saveplot=True, title='dec'+str(n))
    else:
        plot_graph(total_img, [t], order='F', colors=['C2'], saveplot=True, title='dec'+str(n))
    T = resize_graph(T, t.shape)
    T += t